In [ ]:
import lasio
import matplotlib.pyplot as plt
import os

In [ ]:
las_folder = file = r'..\..\..\petrophysical mindboggles\AGS_files\output'

In [ ]:
def find_files(path, recursive=False, extension='', case_sensitive=False):
    '''Function to scan a path (optional recursive) to find all files with a specific
    extension.
    Mandatory argument:
    :path: path/directory to look in for files 
    Optional arguments:
    :recursive: decends into directories. Default = False
    :extension: file extension. Default = ''
    :case_sensitive. Treats extention as case sensitive. Default = False
    Returns a list with all files matching the extension. Files/elements contain the full path.'''
    file_list = list()
    
    for f in os.listdir(path):
        fpath = os.path.join(path,f)
        if os.path.isdir(fpath):
            if recursive == True:
                file_list.extend(find_files(fpath, recursive=recursive, extension=extension, case_sensitive=case_sensitive))
        else:
            if case_sensitive == True:
                if fpath.endswith(extension):
                    file_list.append(fpath)
            else:
                if fpath.lower().endswith(extension.lower()):
                    file_list.append(fpath)
    return(file_list)

In [ ]:
las_files = find_files(las_folder, recursive=False, extension='.las', case_sensitive=False)
las_files

In [ ]:
def create_geotech_plot(file, codes, save=False, log_range_dict=None):
    '''Creates a standard geotech plot
    Mandatory inputs:
    :file: path to LAS file
    :codes: dictionary with codes
    Optional argument:
    :save: False (default) returns plot to the screen. True will save in same location as LAS file
    :log_range_dict: dictionary contain ranges for each curve key = code, value=(min, max, linear/log scale),
         for example: SCPT_RES: (0, 80, 'lin')'''
    
    try:
        las = lasio.read(file)
        df = las.df()
        fig, ax = plt.subplots(nrows=1, ncols=len(df.columns), figsize=(3*len(df.columns),10), sharey=True)
        for i, c in enumerate(df.columns):
            try:
                unit = ''
                for cr in las.curves:
                    if cr.mnemonic == c:
                        unit = cr.unit
                ax[i].plot(df[c], df.index, 'k-')
                if i == 0:
                    ax[i].set_ylabel('DEPTH ('+ [m.unit for m in las.well if m.mnemonic=='STRT'][0] +')', fontweight='bold')
                ax[i].set_xlabel(c + ' (' + [m.unit for m in las.curves if m.mnemonic==c][0] + ')', fontweight='bold')
                if log_range_dict != None:
                    try:
                        xlim = [log_range_dict[c][0], log_range_dict[c][1]]
                        xscale = log_range_dict[c][2]
                        ax[i].set_xlim(xlim)
                        ax[i].set_xscale(xscale)
                    except:
                        pass
            except:
                pass
        ax[0].invert_yaxis()
        plt.suptitle(file+'\n(location: ' + las.well['WELL'].value + ')', fontweight='bold', y=0.95)
        if save == True:
            plt.savefig(file[:-4]+'.png')
        else:
            plt.show()
        plt.close(fig)
    except:
        pass
    

In [ ]:
log_range_dict = {'SCPT_RES': ( 0,    80,  'lin'),
                  'SCPT_FRES':( 0,     1,  'lin'),
                  'SCPT_QT':  (-2,    80,  'lin'),
                  'SCPT_NQT': ( 0,   100,  'lin'),
                  'SCPT_BQ':  (-0.5,   0.5,'lin'),
                  'SCPT_NFR': ( 0,   100,  'lin'),
                  'SCPT_PWP2':(-0.5,   2,  'lin'),
                  'SCDT_PWP2':(-0.5,   2,  'lin'),
                  'SCPP_CIC': ( 0,     5,  'lin')}

In [ ]:
for file in las_files:
    create_geotech_plot(file, list(log_range_dict.keys()), log_range_dict=log_range_dict, save=True)